In [2]:
import pickle
import numpy as np

In [3]:
with open('train_qa.txt','rb') as f:
    train_data = pickle.load(f)

In [4]:
with open('test_qa.txt','rb') as f:
    test_data = pickle.load(f)

In [5]:
type(test_data)

list

In [6]:
#TRAIN-TEST SPLIT ALREADY DONE
#LIST OF TUPLES

In [7]:
' '.join(train_data[0][0])#DATA; QUERY; ANSWER

'Mary moved to the bathroom . Sandra journeyed to the bedroom .'

In [8]:
#CREATE A VOCABULARY

all_data = test_data + train_data

#SET
vocab = set()

for story, question, answer in all_data: #tuple unpacking
    vocab = vocab.union(set(story))
    vocab = vocab.union(set(question))

In [9]:
vocab.add('no')
vocab.add('yes')

In [10]:
vocab

{'.',
 '?',
 'Daniel',
 'Is',
 'John',
 'Mary',
 'Sandra',
 'apple',
 'back',
 'bathroom',
 'bedroom',
 'discarded',
 'down',
 'dropped',
 'football',
 'garden',
 'got',
 'grabbed',
 'hallway',
 'in',
 'journeyed',
 'kitchen',
 'left',
 'milk',
 'moved',
 'no',
 'office',
 'picked',
 'put',
 'the',
 'there',
 'to',
 'took',
 'travelled',
 'up',
 'went',
 'yes'}

In [11]:
#WE ARE LIMITED TO THESE WORDS WHEN ASKING A QUERY
vocab_len = len(vocab) + 1 #PLACEHOLDER WHEN USING KERAS

In [12]:
#HOW LONG IS THE LONGEST STORY AND LONGEST QUESTION

In [13]:
#LONGEST STORY
all_story_lens = [len(data[0]) for data in all_data]

max_story_len = max(all_story_lens)

In [14]:
#LONGEST QUESTION
max_question_len = max([len(data[1]) for data in all_data])

In [15]:
### PART 2: USING KERAS WITH THIS DATA

In [16]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

2023-06-08 02:31:42.418902: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [17]:
from keras.preprocessing.text import Tokenizer

In [18]:
#INSTANTIATE: NO PUNCTUATION FILTERS
tokenizer = Tokenizer(filters=[])

#CREATE DICTIONARY: WORD:ID_NUMBER
tokenizer.fit_on_texts(vocab)

In [19]:
#VECTORISATION FOR STORY, QUESTION, ANSWERS

In [20]:
train_story_text = []
train_question_text = []
train_answers = []

In [21]:
for story, question, answer in train_data:
    train_story_text.append(story)
    train_question_text.append(question)
    train_answers.append(answer)
    

In [22]:
train_story_seq = tokenizer.texts_to_sequences(train_story_text)
train_question_seq = tokenizer.texts_to_sequences(train_question_text)
train_answers_seq = tokenizer.texts_to_sequences(train_answers)

In [23]:
def vectorize_stories(data, word_index=tokenizer.word_index, max_story_len=max_story_len, max_question_len=max_question_len):
    
    #VECTORISE INTO PADDED SEQUENCES
    
    #STORIES
    X = []
    
    #QUESTIONS
    Xq  = []
    
    #CORRECT ANSWER [YES/NO]
    Y = []
    
    for story, query, answer in data:
        #FOR EACH STORY; LIST OF WORD INDICES
        x = [word_index[word.lower()] for word in story]
        
        #FOR EACH QUESTION; LIST OF WORD INDICES
        xq = [word_index[word.lower()] for word in query]
        
        #ANSWER
        y = np.zeros(len(word_index)+1)
        
        #USE NUMPY LOGIC [YES/NO]
        y[word_index[answer]] = 1
        
        X.append(x)
        Xq.append(xq)
        Y.append(y)
        
        
    return (pad_sequences(X, maxlen=max_story_len), pad_sequences(Xq,maxlen=max_question_len), np.array(Y))



In [24]:
inputs_train, queries_train, answers_train = vectorize_stories(train_data)

In [25]:
inputs_test, queries_test, answers_test = vectorize_stories(test_data)

In [26]:
tokenizer.word_index['yes']

18

In [27]:
tokenizer.word_index['no']

34

In [28]:
sum(answers_test)

array([  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0., 497.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0., 503.,   0.,   0.,   0.])

In [29]:
##PART 3: CREATING THE MODEL WITH KERAS

In [30]:
from keras.models import Sequential, Model
from keras.layers.embeddings import Embedding

ModuleNotFoundError: No module named 'keras.layers.embeddings'

In [31]:
from keras.layers import Input, Activation, Dense, Permute, Dropout, add, dot, concatenate, LSTM

In [32]:
from tensorflow.keras.layers import Embedding


In [33]:
#CREATE PLACEHOLDERS USING INPUT

#PLACEHOLDER shape = (max_story_len, batch_size)
input_sequence = Input((max_story_len,))
question = Input((max_question_len,))

In [34]:
vocab_size = vocab_len

In [35]:
#INPUT ENCODER M

#INPUT GETS EMBEDDED TO A SEQUENCE OF VECTORS
input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim=vocab_size, output_dim=64))
input_encoder_m.add(Dropout(0.3))#30% OF NEURONS TURNED OFF TO AVOID OVERFITTING

2023-06-08 02:31:54.748230: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [36]:
#INPUT ENCODER C

#INPUT GETS EMBEDDED TO A SEQUENCE OF VECTORS
input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim=vocab_size, output_dim=max_question_len))
input_encoder_c.add(Dropout(0.3))#30% OF NEURONS TURNED OFF TO AVOID OVERFITTING

In [37]:
#M - > SAMPLES, STORY_MAXLEN, EMBEDDING_DIM
#C -> SAMPLES, STORY_MAXLEN, MAX_QUESTION_LEN
#Q -> SAMPLES, QUERY_MAXLEN, EMBEDDING_DIM

In [38]:
#QUESTION ENCODER
#INPUT GETS EMBEDDED TO A SEQUENCE OF VECTORS
question_encoder = Sequential()
question_encoder.add(Embedding(input_dim=vocab_size, output_dim=64, input_length=max_question_len))
question_encoder.add(Dropout(0.3))#30% OF NEURONS TURNED OFF TO AVOID OVERFITTING

In [39]:
#PASS IN THE PLACEHOLDERS INTO THE ENCODERS


#ENCODED <--- ENCODER(INPUT)
#RESULTS
input_encoded_m = input_encoder_m(input_sequence)
input_encoded_c = input_encoder_c(input_sequence)
question_encoded = question_encoder(question)

In [40]:
match = dot([input_encoded_m,question_encoded], axes = (2,2))
match = Activation('softmax')(match)

In [41]:
#ADD THIS MATCH MATRIX WITH SECOND INPUT VECTOR SEQUENCE
response = add([match, input_encoded_c])
response = Permute((2,1))(response)

In [42]:
#CONCATENATE THE MATCH MATRIX WITH QUESTION VECTOR SEQUENCE
answer = concatenate([response, question_encoded])
answer #CORRECT: (NONE, 6, 220)

<KerasTensor: shape=(None, 6, 220) dtype=float32 (created by layer 'concatenate')>

In [43]:
#REDUCE WITH A RNN: CHOOSE A LSTM
answer = LSTM(32)(answer)

In [44]:
#PREVENT OVERFITTING: DROPOUT
answer = Dropout(0.5)(answer)
answer = Dense(vocab_size)(answer) #SAMPLES,VOCAB_SIZE #YES/NO

In [45]:
#turn into 0/1 with softmax
answer = Activation('softmax')(answer)

In [46]:
model = Model([input_sequence,question],answer)

In [47]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics = ['accuracy'])

In [48]:
##PART - 4: TRAIN, EVALUATE THE MODEL

In [49]:
history = model.fit([inputs_train, queries_train], answers_train, batch_size=32,epochs=250,validation_data = ([inputs_test, queries_test],answers_test))

Epoch 1/250
313/313 [==============================] - 6s 11ms/step - loss: 0.9501 - accuracy: 0.4986 - val_loss: 0.6957 - val_accuracy: 0.4970
Epoch 2/250
313/313 [==============================] - 3s 9ms/step - loss: 0.7111 - accuracy: 0.4999 - val_loss: 0.6947 - val_accuracy: 0.4970
Epoch 3/250
313/313 [==============================] - 3s 8ms/step - loss: 0.6994 - accuracy: 0.5034 - val_loss: 0.6936 - val_accuracy: 0.5030
Epoch 4/250
313/313 [==============================] - 3s 8ms/step - loss: 0.6970 - accuracy: 0.4926 - val_loss: 0.6939 - val_accuracy: 0.5030
Epoch 5/250
313/313 [==============================] - 3s 9ms/step - loss: 0.6962 - accuracy: 0.4923 - val_loss: 0.6932 - val_accuracy: 0.4970
Epoch 6/250
313/313 [==============================] - 3s 9ms/step - loss: 0.6951 - accuracy: 0.5058 - val_loss: 0.6945 - val_accuracy: 0.4970
Epoch 7/250
313/313 [==============================] - 3s 9ms/step - loss: 0.6953 - accuracy: 0.5061 - val_loss: 0.6932 - val_accuracy: 0.503

Epoch 58/250
313/313 [==============================] - 3s 9ms/step - loss: 0.6936 - accuracy: 0.5113 - val_loss: 0.6975 - val_accuracy: 0.5040
Epoch 59/250
313/313 [==============================] - 3s 8ms/step - loss: 0.6932 - accuracy: 0.5150 - val_loss: 0.6977 - val_accuracy: 0.5010
Epoch 60/250
313/313 [==============================] - 3s 9ms/step - loss: 0.6931 - accuracy: 0.5179 - val_loss: 0.6962 - val_accuracy: 0.4840
Epoch 61/250
313/313 [==============================] - 3s 9ms/step - loss: 0.6925 - accuracy: 0.5171 - val_loss: 0.6958 - val_accuracy: 0.4840
Epoch 62/250
313/313 [==============================] - 3s 9ms/step - loss: 0.6916 - accuracy: 0.5303 - val_loss: 0.6964 - val_accuracy: 0.4800
Epoch 63/250
313/313 [==============================] - 3s 9ms/step - loss: 0.6912 - accuracy: 0.5252 - val_loss: 0.6954 - val_accuracy: 0.4870
Epoch 64/250
313/313 [==============================] - 3s 8ms/step - loss: 0.6873 - accuracy: 0.5437 - val_loss: 0.6934 - val_accuracy:

Epoch 115/250
313/313 [==============================] - 3s 9ms/step - loss: 0.3063 - accuracy: 0.8685 - val_loss: 0.4075 - val_accuracy: 0.8150
Epoch 116/250
313/313 [==============================] - 3s 11ms/step - loss: 0.3069 - accuracy: 0.8662 - val_loss: 0.3685 - val_accuracy: 0.8300
Epoch 117/250
313/313 [==============================] - 3s 10ms/step - loss: 0.3074 - accuracy: 0.8699 - val_loss: 0.3789 - val_accuracy: 0.8330
Epoch 118/250
313/313 [==============================] - 3s 10ms/step - loss: 0.3067 - accuracy: 0.8696 - val_loss: 0.3680 - val_accuracy: 0.8260
Epoch 119/250
313/313 [==============================] - 3s 10ms/step - loss: 0.3054 - accuracy: 0.8698 - val_loss: 0.3824 - val_accuracy: 0.8270
Epoch 120/250
313/313 [==============================] - 3s 11ms/step - loss: 0.3017 - accuracy: 0.8670 - val_loss: 0.3858 - val_accuracy: 0.8240
Epoch 121/250
313/313 [==============================] - 4s 13ms/step - loss: 0.3046 - accuracy: 0.8678 - val_loss: 0.4051 - 

313/313 [==============================] - 3s 8ms/step - loss: 0.2653 - accuracy: 0.8849 - val_loss: 0.4448 - val_accuracy: 0.8190
Epoch 172/250
313/313 [==============================] - 3s 8ms/step - loss: 0.2677 - accuracy: 0.8857 - val_loss: 0.4058 - val_accuracy: 0.8380
Epoch 173/250
313/313 [==============================] - 3s 8ms/step - loss: 0.2679 - accuracy: 0.8870 - val_loss: 0.4189 - val_accuracy: 0.8320
Epoch 174/250
313/313 [==============================] - 3s 11ms/step - loss: 0.2621 - accuracy: 0.8885 - val_loss: 0.4385 - val_accuracy: 0.8290
Epoch 175/250
313/313 [==============================] - 4s 13ms/step - loss: 0.2677 - accuracy: 0.8870 - val_loss: 0.4656 - val_accuracy: 0.8230
Epoch 176/250
313/313 [==============================] - 4s 12ms/step - loss: 0.2608 - accuracy: 0.8880 - val_loss: 0.4280 - val_accuracy: 0.8220
Epoch 177/250
313/313 [==============================] - 4s 13ms/step - loss: 0.2676 - accuracy: 0.8857 - val_loss: 0.4229 - val_accuracy: 0.

313/313 [==============================] - 3s 9ms/step - loss: 0.2342 - accuracy: 0.9005 - val_loss: 0.4873 - val_accuracy: 0.8210
Epoch 228/250
313/313 [==============================] - 3s 9ms/step - loss: 0.2309 - accuracy: 0.9024 - val_loss: 0.5630 - val_accuracy: 0.8180
Epoch 229/250
313/313 [==============================] - 3s 9ms/step - loss: 0.2308 - accuracy: 0.9039 - val_loss: 0.5955 - val_accuracy: 0.8140
Epoch 230/250
313/313 [==============================] - 3s 9ms/step - loss: 0.2251 - accuracy: 0.9066 - val_loss: 0.5219 - val_accuracy: 0.8220
Epoch 231/250
313/313 [==============================] - 3s 9ms/step - loss: 0.2198 - accuracy: 0.9079 - val_loss: 0.5369 - val_accuracy: 0.8220
Epoch 232/250
313/313 [==============================] - 3s 9ms/step - loss: 0.2212 - accuracy: 0.9072 - val_loss: 0.5292 - val_accuracy: 0.8230
Epoch 233/250
313/313 [==============================] - 3s 9ms/step - loss: 0.2268 - accuracy: 0.9064 - val_loss: 0.5576 - val_accuracy: 0.8060

In [50]:
#EVALUATING ON GIVEN TEST SET


#PREDICT RESULTS
pred_results = history.predict(([inputs_test, queries_test]))

AttributeError: 'History' object has no attribute 'predict'

In [ ]:
#test_data -> list of tuples [story, question, answer]
#pred_results has probabilities for every single word

#max probability
val_max = np.argmax(pred_results[0])

#print the word
for key, val in tokenizer.word_index.items():
    if val == val_max:
        k = key

In [ ]:
#ANSWER = 
k

In [ ]:
#PROBABILITY OF BEING SURE OF THAT ANSWER
pred_results[0][val_max]

In [ ]:
#FOR YOUR OWN STORIES; YOU CAN ONLY USE WORDS FROM THAT VOCABULARY

#SPACE BEFORE AND AFTER PUNCTUATION: SAME FORMAT AS TRAIN_DATA
my_story = "John left the kitches . Sandra dropped the football in tthe garden ."

In [ ]:
my_question = 'Is the football in the garden ?' #SAME FORMAT AS TRAINING DATA

In [ ]:
my_data = [(my_story.split(), my_question.split(), 'yes')] 

In [ ]:
#VECTORIZED
my_story, my_ques, my_ans = vectorize_stories(my_data)

In [ ]:
#PREDICT ONLY AFTER STORY AND QUESTION
pred_results = model.predict(([my_story, my_ques]))

In [ ]:
val_max = np.argmax(pred_results[0])

In [ ]:
for key, val in tokenizer.word_index.items():
    if val == val_max:
        k = key

In [ ]:
k

In [ ]:
pred_results[0][val_max]